# Importaciones

In [ ]:
import ee

# Trigger the authentication flow.
ee.Authenticate()

# Initialize the library.
ee.Initialize()
import datetime
import matplotlib.pyplot as plt
import csv
import pandas as pd
from google.colab import drive
drive.mount("/content/gdrive")
import numpy as np
import matplotlib.image as mpimg
import os
import cv2
import time
import random
import pandas as pd
from funciones_google_earth.py import maskS2clouds,\
  incendio, temperatura, humedad_relativa , poblacion,\
  viento

# 1- Lectura ciudades

El fichero recibido por el CE no incorpora las coordenadas X e Y por lo que a partir de las ciudades y el país usaré la librería de *geopy* para recibir las coordenadas que dejen en su centro la ciudad.

No se pueden realizar consultas superiores a 1.000 la hora por lo que partiré en 4 el dataset original

In [ ]:
df = pd.read_csv ('/content/gdrive/My Drive/incendios_satelite/historia incendios.csv', sep=';', encoding='latin1')

In [ ]:
df1 = df.iloc[0:900,:]
df2 = df.iloc[900:1800,:]
df3 = df.iloc[1800:2700,:]
df4 = df.iloc[2700:3600,:]

In [ ]:
from geopy import geocoders
gn = geocoders.GeoNames(username="cpintorv")
df1["coordenadas"] = df1.apply(lambda x: gn.geocode(str(x.admlvl5) + " , " + str(x.country)), axis=1)

In [ ]:
df2["coordenadas"] = df2.apply(lambda x: gn.geocode(str(x.admlvl5) + " , " + str(x.country)), axis=1)

In [ ]:
df3["coordenadas"] = df3.apply(lambda x: gn.geocode(str(x.admlvl5) + " , " + str(x.country)), axis=1)   

In [ ]:
df4["coordenadas"] = df4.apply(lambda x: gn.geocode(str(x.admlvl5) + " , " + str(x.country)), axis=1)

In [ ]:
df1 = df1[df1["coordenadas"].notnull()]
df2 = df2[df2["coordenadas"].notnull()]
df3 = df3[df3["coordenadas"].notnull()]
df4 = df4[df4["coordenadas"].notnull()]

In [ ]:
# Limpiamos las variables que no vaomos a usar en las llamadas
# df1
df1["x"] = df1["coordenadas"].apply(lambda x: x[1][0])
df1["y"] = df1["coordenadas"].apply(lambda x: x[1][1])
df1.drop("coordenadas", axis=1, inplace=True)
df1.drop("admlvl1", axis=1, inplace=True)
df1.drop("country", axis=1, inplace=True)
df1.drop("eu_area", axis=1, inplace=True)
df1.drop("area_code", axis=1, inplace=True)
df1.drop("updated", axis=1, inplace=True)
df1.drop("noneu", axis=1, inplace=True)
df1.drop("X", axis=1, inplace=True)
df1.drop("Y", axis=1, inplace=True)
df1.drop("area_ha", axis=1, inplace=True)
df1.drop("iso2", axis=1, inplace=True)
df1.drop("iso3", axis=1, inplace=True)
df1.drop("admlvl2", axis=1, inplace=True)
df1.drop("admlvl3", axis=1, inplace=True)
df1.drop("admlvl5", axis=1, inplace=True)
df1.drop("map_source", axis=1, inplace=True)
df1.drop("broadleaved_forest_percent", axis=1, inplace=True)
df1.drop("coniferous_forest_percent", axis=1, inplace=True)
df1.drop("mixed_forest_percent", axis=1, inplace=True)
df1.drop("sclerophillous_vegetation_percent", axis=1, inplace=True)
df1.drop("transitional_vegetation_percent", axis=1, inplace=True)
df1.drop("other_natural_percent", axis=1, inplace=True)
df1.drop("agriculture_percent", axis=1, inplace=True)
df1.drop("artificial_percent", axis=1, inplace=True)
df1.drop("other_percent", axis=1, inplace=True)
df1.drop("natura2k_percent", axis=1, inplace=True)

# df2
df2["x"] = df2["coordenadas"].apply(lambda x: x[1][0])
df2["y"] = df2["coordenadas"].apply(lambda x: x[1][1])
df2.drop("coordenadas", axis=1, inplace=True)
df2.drop("admlvl1", axis=1, inplace=True)
df2.drop("country", axis=1, inplace=True)
df2.drop("eu_area", axis=1, inplace=True)
df2.drop("area_code", axis=1, inplace=True)
df2.drop("updated", axis=1, inplace=True)
df2.drop("noneu", axis=1, inplace=True)
df2.drop("X", axis=1, inplace=True)
df2.drop("Y", axis=1, inplace=True)
df2.drop("area_ha", axis=1, inplace=True)
df2.drop("iso2", axis=1, inplace=True)
df2.drop("iso3", axis=1, inplace=True)
df2.drop("admlvl2", axis=1, inplace=True)
df2.drop("admlvl3", axis=1, inplace=True)
df2.drop("admlvl5", axis=1, inplace=True)
df2.drop("map_source", axis=1, inplace=True)
df2.drop("broadleaved_forest_percent", axis=1, inplace=True)
df2.drop("coniferous_forest_percent", axis=1, inplace=True)
df2.drop("mixed_forest_percent", axis=1, inplace=True)
df2.drop("sclerophillous_vegetation_percent", axis=1, inplace=True)
df2.drop("transitional_vegetation_percent", axis=1, inplace=True)
df2.drop("other_natural_percent", axis=1, inplace=True)
df2.drop("agriculture_percent", axis=1, inplace=True)
df2.drop("artificial_percent", axis=1, inplace=True)
df2.drop("other_percent", axis=1, inplace=True)
df2.drop("natura2k_percent", axis=1, inplace=True)

# df3
df3["x"] = df3["coordenadas"].apply(lambda x: x[1][0])
df3["y"] = df3["coordenadas"].apply(lambda x: x[1][1])
df3.drop("coordenadas", axis=1, inplace=True)
df3.drop("admlvl1", axis=1, inplace=True)
df3.drop("country", axis=1, inplace=True)
df3.drop("eu_area", axis=1, inplace=True)
df3.drop("area_code", axis=1, inplace=True)
df3.drop("updated", axis=1, inplace=True)
df3.drop("noneu", axis=1, inplace=True)
df3.drop("X", axis=1, inplace=True)
df3.drop("Y", axis=1, inplace=True)
df3.drop("area_ha", axis=1, inplace=True)
df3.drop("iso2", axis=1, inplace=True)
df3.drop("iso3", axis=1, inplace=True)
df3.drop("admlvl2", axis=1, inplace=True)
df3.drop("admlvl3", axis=1, inplace=True)
df3.drop("admlvl5", axis=1, inplace=True)
df3.drop("map_source", axis=1, inplace=True)
df3.drop("broadleaved_forest_percent", axis=1, inplace=True)
df3.drop("coniferous_forest_percent", axis=1, inplace=True)
df3.drop("mixed_forest_percent", axis=1, inplace=True)
df3.drop("sclerophillous_vegetation_percent", axis=1, inplace=True)
df3.drop("transitional_vegetation_percent", axis=1, inplace=True)
df3.drop("other_natural_percent", axis=1, inplace=True)
df3.drop("agriculture_percent", axis=1, inplace=True)
df3.drop("artificial_percent", axis=1, inplace=True)
df3.drop("other_percent", axis=1, inplace=True)
df3.drop("natura2k_percent", axis=1, inplace=True)

# df4
df4["x"] = df4["coordenadas"].apply(lambda x: x[1][0])
df4["y"] = df4["coordenadas"].apply(lambda x: x[1][1])
df4.drop("coordenadas", axis=1, inplace=True)
df4.drop("admlvl1", axis=1, inplace=True)
df4.drop("country", axis=1, inplace=True)
df4.drop("eu_area", axis=1, inplace=True)
df4.drop("area_code", axis=1, inplace=True)
df4.drop("updated", axis=1, inplace=True)
df4.drop("noneu", axis=1, inplace=True)
df4.drop("X", axis=1, inplace=True)
df4.drop("Y", axis=1, inplace=True)
df4.drop("area_ha", axis=1, inplace=True)
df4.drop("iso2", axis=1, inplace=True)
df4.drop("iso3", axis=1, inplace=True)
df4.drop("admlvl2", axis=1, inplace=True)
df4.drop("admlvl3", axis=1, inplace=True)
df4.drop("admlvl5", axis=1, inplace=True)
df4.drop("map_source", axis=1, inplace=True)
df4.drop("broadleaved_forest_percent", axis=1, inplace=True)
df4.drop("coniferous_forest_percent", axis=1, inplace=True)
df4.drop("mixed_forest_percent", axis=1, inplace=True)
df4.drop("sclerophillous_vegetation_percent", axis=1, inplace=True)
df4.drop("transitional_vegetation_percent", axis=1, inplace=True)
df4.drop("other_natural_percent", axis=1, inplace=True)
df4.drop("agriculture_percent", axis=1, inplace=True)
df4.drop("artificial_percent", axis=1, inplace=True)
df4.drop("other_percent", axis=1, inplace=True)
df4.drop("natura2k_percent", axis=1, inplace=True)

In [ ]:
df_final = pd.concat([df1, df2, df3, df4])

In [ ]:
df_final_sorted.head()

In [ ]:
df_final_sorted["initialdate"] = pd.to_datetime(df_final_sorted["initialdate"])
df_final_sorted["finaldate"] = pd.to_datetime(df_final_sorted["finaldate"])
df_final_sorted = df_final_sorted.sort_values(by=["x", "initialdate"])
df_final_sorted = df_final_sorted.drop_duplicates(subset = 'x', keep = 'first')

In [ ]:
df_final_sorted["year"] = df_final_sorted["initialdate"].apply(lambda x: x.year)
df_final_sorted["month"] = df_final_sorted["initialdate"].apply(lambda x: x.month)
df_final_sorted["day"] = df_final_sorted["initialdate"].apply(lambda x: x.day)

In [ ]:
# Exportamos a CSV para poder quitar los duplicados en excel
df_final_sorted.to_csv('/content/gdrive/My Drive/incendios_satelite/historia incendios_limpio.csv')

# 2-Imágenes reales target

In [ ]:
path_incendios = '/content/gdrive/My Drive/incendios_satelite/' # Los dejo
# en esta ruta y los iré moviendo a la carpeta correspondiente
lista_ficheros = []
lista_id = []
df_final_sorted = pd.read_csv(
    '/content/gdrive/My Drive/incendios_satelite/historia incendios_limpio.csv', sep=',')
for file in os.listdir(path_incendios):
  if "incendio" in file:
    lista_ficheros.append(int(file.split("i")[0]))

df_existentes = pd.DataFrame(data = lista_ficheros, columns = ['id'])
df_satelite2 = pd.merge(df_final_sorted,
                        df_existentes,
                        how = 'inner',
                        left_on = 'id',
                        right_on = 'id')

In [ ]:
df_satelite2.apply(lambda x: temperatura(id = x["id"],
                                     x = x["x"],
                                     y = x["y"],
                                     year = x["year"],
                                     month = x["month"],
                                     day = x["day"],
                                     anticipacion = 1), axis=1)

In [ ]:
df_satelite2.apply(lambda x: humedad_relativa(id = x["id"],
                                     x = x["x"],
                                     y = x["y"],
                                     year = x["year"],
                                     month = x["month"],
                                     day = x["day"],
                                     anticipacion = 1), axis=1)

In [ ]:
df_satelite2.apply(lambda x: poblacion(id = x["id"],
                                     x = x["x"],
                                     y = x["y"],
                                     year = x["year"],
                                     month = x["month"],
                                     day = x["day"],
                                     anticipacion = 1), axis=1)

In [ ]:
df_satelite2.apply(lambda x: viento(id = x["id"],
                                     x = x["x"],
                                     y = x["y"],
                                     year = x["year"],
                                     month = x["month"],
                                     day = x["day"],
                                     anticipacion = 1,
                                    direccion='U'), axis=1)

In [ ]:
df_satelite2.apply(lambda x: viento(id = x["id"],
                                     x = x["x"],
                                     y = x["y"],
                                     year = x["year"],
                                     month = x["month"],
                                     day = x["day"],
                                     anticipacion = 1,
                                    direccion='V'), axis=1)

# 3-Imágenes reales control

Las imágenes de control las he generado en excel manteniendo la distribución de año y mes que se había seguido en la generación de imágenes target

In [ ]:
path_incendios = '/content/gdrive/My Drive/incendios_satelite/'
df_satelite_control = pd.read_csv (path_incendios + 'Control.csv', sep=';', encoding='latin1')
len(df_satelite_control)

In [ ]:
df_satelite_control.apply(lambda x: incendio(id = x["id"],
                                     x = x["X"],
                                     y = x["Y"],
                                     year = x["year"],
                                     month = x["month"],
                                     day = x["day"],
                                     anticipacion = 1), axis=1)

# 4-Genera dataframe temperatura, humedad relativa, población y viento

In [ ]:
# Genero un dataframe con los "id" de las imágenes de incendios sobre las
# que se ha generado imagen real. Se aplica para control, para target ya
# se ha eliminado en el excel
path_incendios = '/content/gdrive/My Drive/incendios_satelite/Control'
lista_ficheros = []
lista_id = []
for file in os.listdir(path_incendios):
  if "incendio" in file:
    lista_ficheros.append(int(file.split("i")[0]))

df_existentes = pd.DataFrame(data = lista_ficheros, columns = ['id'])

# Leo el csv con todos los casos de control
df_satelite_control = pd.read_csv (path_incendios + 'Total casos2.csv', sep=';',
                                   encoding='latin1')

# Cruzo ambos dataframes de manera que tenga únicamente los que sí ha conseguido
# generar la imagen real
df_existentes_control = pd.merge(df_satelite_control, df_existentes, 
                                 how = 'inner', left_on = 'id', 
                                 right_on = 'id')

# Exportamos para quitar los febreros con 29 y 30 días y volvemos a leer
df_existentes_control.to_csv(path_incendios + 'total_incendios_control.csv', 
                             index = False)

# 5-Temperatura

## 5.1- Control

In [ ]:
# Leo el fichero con los incendios de control
path_incencios_csv = '/content/gdrive/My Drive/incendios_satelite/'
df_satelite_target = pd.read_csv (
    path_incencios_csv + 'total_incendios_control.csv', sep=',', encoding='latin1')
print("Tamaño de control: {}". format(len(df_satelite_target)))

In [ ]:
# Llamadas a la función de temperatura
df_existentes_control2.apply(lambda x: temperatura(id = x["id"],
                                     x = x["X"],
                                     y = x["Y"],
                                     year = x["year"],
                                     month = x["month"],
                                     day = x["day"],
                                     anticipacion = 1), axis=1)

## 5.2-Target

In [ ]:
# Leo el fichero con los incendios de target
path_incencios_csv = '/content/gdrive/My Drive/incendios_satelite/'
df_satelite_target = pd.read_csv (path_incencios_csv + '/incendios_target3.csv',
                                  sep=',', encoding='latin1')

In [ ]:
# Llamadas a la función de temperatura
df_satelite_target.apply(lambda x: temperatura(id = x["id"],
                                     x = x["X"],
                                     y = x["Y"],
                                     year = x["year"],
                                     month = x["month"],
                                     day = x["day"],
                                     anticipacion = 1), axis=1)

# 6-Humedad relativa

## 6.1-Control

In [ ]:
# Leo el fichero con los incendios de control
path_incencios_csv = '/content/gdrive/My Drive/incendios_satelite/'
df_satelite_target = pd.read_csv (
    path_incencios_csv + 'total_incendios_control.csv', sep=',', encoding='latin1')
print("Tamaño de control: {}". format(len(df_satelite_target)))

In [ ]:
# Llamadas a la función de humedad relativa
df_satelite_target.apply(lambda x: humedad_relativa(id = x["id"],
                                     x = x["X"],
                                     y = x["Y"],
                                     year = x["year"],
                                     month = x["month"],
                                     day = x["day"],
                                     anticipacion = 1), axis=1)

## 6.2-Target

In [ ]:
# Leo el fichero con los incendios de target
path_incencios_csv = '/content/gdrive/My Drive/incendios_satelite/'
df_satelite_target = pd.read_csv (path_incencios_csv + '/incendios_target3.csv',
                                  sep=',', encoding='latin1')
print(len(df_satelite_target))
df_satelite_target.head()

In [ ]:
# Llamadas a la función de humedad relativa
df_satelite_target.apply(lambda x: humedad_relativa(id = x["id"],
                                     x = x["X"],
                                     y = x["Y"],
                                     year = x["year"],
                                     month = x["month"],
                                     day = x["day"],
                                     anticipacion = 1), axis=1)

In [ ]:
# Número de ficheros leídos
lista_ficheros = []
for file in os.listdir(path_incencios_csv + 'Incendios masivos'):
  if "ND" in file:
    lista_ficheros.append(int(file.split("N")[0]))
len(lista_ficheros)

# 7.Población

## 7.1- Control

In [ ]:
# Leo el fichero con los incendios de control
path_incencios_csv = '/content/gdrive/My Drive/incendios_satelite/'
df_satelite_target = pd.read_csv (path_incencios_csv + 'total_incendios_control.csv', sep=',', encoding='latin1')
print("Tamaño de control: {}". format(len(df_satelite_target)))

In [ ]:
# Llamada a la función de población
df_satelite_target.apply(lambda x: poblacion(id = x["id"],
                                     x = x["X"],
                                     y = x["Y"],
                                     year = x["year"],
                                     month = x["month"],
                                     day = x["day"],
                                     anticipacion = 1), axis=1)

## 7.2-Población de los Target

In [ ]:
# Leo el fichero con los incendios target
path_incencios_csv = '/content/gdrive/My Drive/incendios_satelite/'
df_satelite_target = pd.read_csv (path_incencios_csv + '/incendios_target3.csv', sep=',', encoding='latin1')
print(len(df_satelite_target))
df_satelite_target.head()

In [ ]:
# Llamada a la función de población
df_satelite_target.apply(lambda x: poblacion(id = x["id"],
                                     x = x["X"],
                                     y = x["Y"],
                                     year = x["year"],
                                     month = x["month"],
                                     day = x["day"],
                                     anticipacion = 1), axis=1)

# 8-Viento

In [ ]:
# Leo el fichero con los casos control
path_incencios_csv = '/content/gdrive/My Drive/incendios_satelite/'
df_satelite_target = pd.read_csv (
    path_incencios_csv + 'total_incendios_control.csv', sep=',', encoding='latin1')
print("Tamaño de control: {}". format(len(df_satelite_target)))

In [ ]:
# Leo el fichero con los casos target
path_incencios_csv = '/content/gdrive/My Drive/incendios_satelite/'
df_satelite_target = pd.read_csv (
    path_incencios_csv + '/incendios_target3.csv', sep=',', encoding='latin1')

In [ ]:
# Llamo a la función de tiempo
df_satelite_target.apply(lambda x: viento(id = x["id"],
                                     x = x["X"],
                                     y = x["Y"],
                                     year = x["year"],
                                     month = x["month"],
                                     day = x["day"],
                                     anticipacion = 1,
                                     direccion="U"), axis=1)